# Layer Analytics
This notebook contains code to parse a saved TensorFlow model file and generate a pandas dataframe that contains information about all the layers.

In [53]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
import plotly.express as px

# enable plotly in VS Studio Code
import plotly.io as pio
pio.renderers.default = "notebook_connected"

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

from workbench.utils.utils import create_filepaths, parse_model_name
from workbench.config.config import initialize


In [2]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [3]:
#model_name = "mobilenetv1_0.1_224_c3_o3_000"
model_name = "mobilenetv2_0.1_96_c3_o3_keras"

In [4]:
parse_model_name(model_name)

('mobilenetv2', '0.1', '96', 'c3', 'o3', 'keras')

In [5]:
models_dir = initialize()

In [6]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models
i:\tinyml\tiny_cnn\models\mobilenetv2_0.1_96_c3_o3_keras does not exist.
Created path: i:\tinyml\tiny_cnn\models\mobilenetv2_0.1_96_c3_o3_keras.


In [10]:
filepath = models_dir.joinpath(f"{model_name}/{model_name}_layers.pkl")
#filepath = f"i:/tinyml/tiny_cnn/models/{model_name}/{model_name}_layers.pkl"
df = pd.read_pickle(filepath)

In [11]:
df.columns

Index(['index', 'name', 'layer_type', 'input_shape', 'input', 'output_shape',
       'dtype', 'params', 'variables', 'weights_shape', 'weights_name',
       'weights_dtype', '#_activations', 'OPS', 'MACS', 'filters', 'trainable',
       'kernel_size', 'strides', 'padding', 'dilation_rate', 'groups',
       'activation', 'use_bias', 'data_format'],
      dtype='object')

In [12]:
df.head(10)

,index,name,layer_type,input_shape,input,output_shape,dtype,params,variables,weights_shape,weights_name,weights_dtype,#_activations,OPS,MACS,filters,trainable,kernel_size,strides,padding,dilation_rate,groups,activation,use_bias,data_format
0,0,input_1,<class 'keras.engine.input_layer.InputLayer'>,"[(None, 96, 96, 3)]","KerasTensor(type_spec=TensorSpec(shape=(None, ...","[(None, 96, 96, 3)]",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Conv1,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 96, 96, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,216,1,"(3, 3, 3, 8)",Conv1/kernel:0,<dtype: 'float32'>,18432,995328,497664,8,True,"(3, 3)","(2, 2)",same,"(1, 1)",1,linear,False,channels_last
2,2,bn_Conv1,<class 'keras.layers.normalization.batch_norma...,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,32,4,(8),bn_Conv1/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,Conv1_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,expanded_conv_depthwise,<class 'keras.layers.convolutional.depthwise_c...,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,72,1,"(3, 3, 8, 1)",expanded_conv_depthwise/depthwise_kernel:0,<dtype: 'float32'>,18432,331776,165888,0,True,"(3, 3)","(1, 1)",same,"(1, 1)",1,linear,False,channels_last
5,5,expanded_conv_depthwise_BN,<class 'keras.layers.normalization.batch_norma...,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,32,4,(8),expanded_conv_depthwise_BN/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0,0
6,6,expanded_conv_depthwise_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,7,expanded_conv_project,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,64,1,"(1, 1, 8, 8)",expanded_conv_project/kernel:0,<dtype: 'float32'>,18432,294912,147456,8,True,"(1, 1)","(1, 1)",same,"(1, 1)",1,linear,False,channels_last
8,8,expanded_conv_project_BN,<class 'keras.layers.normalization.batch_norma...,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,32,4,(8),expanded_conv_project_BN/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0,0
9,9,expanded_conv_add,<class 'keras.layers.merging.add.Add'>,"[(None, 48, 48, 8), (None, 48, 48, 8)]","[KerasTensor(type_spec=TensorSpec(shape=(None,...","(None, 48, 48, 8)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Helper functions

In [13]:
def split_tuples(x):
    x = str(x).strip("()")
    x = x.split(",")
    return x

In [14]:
def return_batch_size(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)
    return split_strings[0].strip()

In [15]:
def return_input_height(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [3,4]:
        return split_strings[1].strip()
    else:
        return "None"


In [16]:
def return_input_width(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [4]:
        return split_strings[2].strip()
    else:
        return "None"


In [17]:
def return_channels(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)

    return split_strings[-1].strip()

In [18]:
def split_shape_columns(df, colum_name):
    df[colum_name] = df[colum_name].map(str)
    df[colum_name] = df[colum_name].str.strip("[]").replace('\)\,', ')*', regex=True)
    df_cols = df[colum_name].str.split("*", expand=True)
    prefix = colum_name.split("_")[0]
    df_cols.columns = [f"{prefix}_{x+1}" for x in df_cols.columns]
    return df_cols

# Input shapes & activations

In [19]:
df_input = split_shape_columns(df, "input_shape")
df_input

,input_1,input_2
0,"(None, 96, 96, 3)",None
1,"(None, 96, 96, 3)",None
2,"(None, 48, 48, 8)",None
3,"(None, 48, 48, 8)",None
4,"(None, 48, 48, 8)",None
...,...,...
152,"(None, 3, 3, 32)",None
153,"(None, 3, 3, 1280)",None
154,"(None, 3, 3, 1280)",None
155,"(None, 3, 3, 1280)",None


In [20]:
# #df["input_shape"][0] =(None, None, None, None)
# df["input_shape"] = df["input_shape"].map(str)
# df["input_shape"] = df["input_shape"].str.strip("[]").replace('\)\,', ')*', regex=True)
# df_input = df["input_shape"].str.split("*", expand=True)
# df_input.columns =[f"input_{x+1}" for x in df_input.columns]
# no_input_cols = len(df_input.columns)
# print(no_input_cols)
# df_input


In [21]:
#df_input["input_1_split"] = df_input["input_1"].apply(split_tuples)

In [22]:
no_input_cols = len(df_input.columns)

In [23]:
for input in range(1,no_input_cols+1):
    df_input[f"b_i_{input}"]= df_input[f"input_{input}"].apply(return_batch_size)
    df_input[f"h_i_{input}"] = df_input[f"input_{input}"].apply(return_input_height)
    df_input[f"w_i_{input}"] = df_input[f"input_{input}"].apply(return_input_width)
    df_input[f"c_i_{input}"] = df_input[f"input_{input}"].apply(return_channels)

df_input

,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2
0,"(None, 96, 96, 3)",None,None,96,96,3,None,None,None,None
1,"(None, 96, 96, 3)",None,None,96,96,3,None,None,None,None
2,"(None, 48, 48, 8)",None,None,48,48,8,None,None,None,None
3,"(None, 48, 48, 8)",None,None,48,48,8,None,None,None,None
4,"(None, 48, 48, 8)",None,None,48,48,8,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
152,"(None, 3, 3, 32)",None,None,3,3,32,None,None,None,None
153,"(None, 3, 3, 1280)",None,None,3,3,1280,None,None,None,None
154,"(None, 3, 3, 1280)",None,None,3,3,1280,None,None,None,None
155,"(None, 3, 3, 1280)",None,None,3,3,1280,None,None,None,None


In [57]:
input_cols = df_input.columns.to_list()


True

In [24]:
#df_input["w_i_2"].replace('None', np.nan, inplace=True)
df_input["b_i_1"].replace('None', np.nan, inplace=True)
df_input["w_i_1"].replace('None', np.nan, inplace=True)
df_input["h_i_1"].replace('None', np.nan, inplace=True)
df_input["c_i_1"].replace('None', np.nan, inplace=True)

if "b_i_2" in input_cols:
    df_input["b_i_2"] = df_input["b_i_2"].str.replace("\(", "")
    df_input["b_i_2"].replace('None', np.nan, inplace=True)
    df_input["w_i_2"].replace('None', np.nan, inplace=True)
    df_input["h_i_2"].replace('None', np.nan, inplace=True)
    df_input["c_i_2"].replace('None', np.nan, inplace=True)
df_input.fillna(1, inplace=True)


C:\Users\Susanne\AppData\Local\Temp\ipykernel_21412\2502296835.py:7: FutureWarning:

The default value of regex will change from True to False in a future version.



In [25]:
df_input

,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2
0,"(None, 96, 96, 3)",1,1.0,96,96,3,1.0,1,1,1
1,"(None, 96, 96, 3)",1,1.0,96,96,3,1.0,1,1,1
2,"(None, 48, 48, 8)",1,1.0,48,48,8,1.0,1,1,1
3,"(None, 48, 48, 8)",1,1.0,48,48,8,1.0,1,1,1
4,"(None, 48, 48, 8)",1,1.0,48,48,8,1.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
152,"(None, 3, 3, 32)",1,1.0,3,3,32,1.0,1,1,1
153,"(None, 3, 3, 1280)",1,1.0,3,3,1280,1.0,1,1,1
154,"(None, 3, 3, 1280)",1,1.0,3,3,1280,1.0,1,1,1
155,"(None, 3, 3, 1280)",1,1.0,3,3,1280,1.0,1,1,1


In [26]:
#df_input.c_i_1.value_counts()

In [27]:

df_input["w_i_1"] = df_input["w_i_1"].map(int)
df_input["h_i_1"] = df_input["h_i_1"].map(int)
df_input["b_i_1"] = df_input["b_i_1"].map(int)
df_input["c_i_1"] = df_input["c_i_1"].map(int)

if "b_i_2" in input_cols:
    df_input["b_i_2"] = df_input["b_i_2"].map(int)
    df_input["w_i_2"] = df_input["w_i_2"].map(int)
    df_input["h_i_2"] = df_input["h_i_2"].map(int)
    df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [28]:

df_input["#i_1_activations"] = df_input["h_i_1"] * df_input["w_i_1"] * df_input["c_i_1"]


In [29]:
def calculate_i_2_activations(x):
    h_i = int(x["h_i_2"])#.map(int)
    w_i = int(x["w_i_2"])#.map(int)
    c_i = int(x["c_i_2"])#.map(int)
    out = h_i * w_i * c_i
    if out >1:
        return out
    else:
        return 0

In [30]:
if "b_i_2" in input_cols:
    df_input["#i_2_activations"] = df_input.apply(calculate_i_2_activations, axis=1)

In [31]:

# df_input["b_i_1"] = df_input["b_i_1"].map(int)
# df_input["w_i_1"] = df_input["w_i_1"].map(int)
# df_input["h_i_1"] = df_input["h_i_1"].map(int)
# df_input["c_i_1"] = df_input["c_i_1"].map(int)

# df_input["b_i_2"] = df_input["b_i_2"].map(int)
# df_input["w_i_2"] = df_input["w_i_2"].map(int)
# df_input["h_i_2"] = df_input["h_i_2"].map(int)
# df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [32]:
df_input

,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2,#i_1_activations,#i_2_activations
0,"(None, 96, 96, 3)",1,1,96,96,3,1,1,1,1,27648,0
1,"(None, 96, 96, 3)",1,1,96,96,3,1,1,1,1,27648,0
2,"(None, 48, 48, 8)",1,1,48,48,8,1,1,1,1,18432,0
3,"(None, 48, 48, 8)",1,1,48,48,8,1,1,1,1,18432,0
4,"(None, 48, 48, 8)",1,1,48,48,8,1,1,1,1,18432,0
...,...,...,...,...,...,...,...,...,...,...,...,...
152,"(None, 3, 3, 32)",1,1,3,3,32,1,1,1,1,288,0
153,"(None, 3, 3, 1280)",1,1,3,3,1280,1,1,1,1,11520,0
154,"(None, 3, 3, 1280)",1,1,3,3,1280,1,1,1,1,11520,0
155,"(None, 3, 3, 1280)",1,1,3,3,1280,1,1,1,1,11520,0


In [33]:
#df_input["#i_2_activations"].value_counts()

0        146
288        3
1152       2
576        2
144        2
18432      1
4608       1
Name: #i_2_activations, dtype: int64

In [34]:
df = df.join(df_input)

# Output shape and activations

In [35]:
df_output = split_shape_columns(df, "output_shape")
df_output

,output_1
0,"(None, 96, 96, 3)"
1,"(None, 48, 48, 8)"
2,"(None, 48, 48, 8)"
3,"(None, 48, 48, 8)"
4,"(None, 48, 48, 8)"
...,...
152,"(None, 3, 3, 1280)"
153,"(None, 3, 3, 1280)"
154,"(None, 3, 3, 1280)"
155,"(None, 1280)"


In [36]:
no_output_cols = len(df_output.columns)
no_output_cols

1

In [37]:
for output in range(1,no_output_cols+1):
    df_output[f"b_o_{output}"]= df_output[f"output_{output}"].apply(return_batch_size)
    df_output[f"h_o_{output}"] = df_output[f"output_{output}"].apply(return_input_height)
    df_output[f"w_o_{output}"] = df_output[f"output_{output}"].apply(return_input_width)
    df_output[f"c_o_{output}"] = df_output[f"output_{output}"].apply(return_channels)

df_output

,output_1,b_o_1,h_o_1,w_o_1,c_o_1
0,"(None, 96, 96, 3)",None,96,96,3
1,"(None, 48, 48, 8)",None,48,48,8
2,"(None, 48, 48, 8)",None,48,48,8
3,"(None, 48, 48, 8)",None,48,48,8
4,"(None, 48, 48, 8)",None,48,48,8
...,...,...,...,...,...
152,"(None, 3, 3, 1280)",None,3,3,1280
153,"(None, 3, 3, 1280)",None,3,3,1280
154,"(None, 3, 3, 1280)",None,3,3,1280
155,"(None, 1280)",None,None,None,1280


In [38]:
df_output["b_o_1"].replace('None', np.nan, inplace=True)
df_output["w_o_1"].replace('None', np.nan, inplace=True)
df_output["h_o_1"].replace('None', np.nan, inplace=True)
df_output["c_o_1"].replace('None', np.nan, inplace=True)
df_output.fillna(1, inplace=True)

In [39]:
df_output["w_o_1"] = df_output["w_o_1"].map(int)
df_output["h_o_1"] = df_output["h_o_1"].map(int)
df_output["b_o_1"] = df_output["b_o_1"].map(int)
df_output["c_o_1"] = df_output["c_o_1"].map(int)

In [40]:
df_output["#o_1_activations"] = df_output["h_o_1"] * df_output["w_o_1"] * df_output["c_o_1"]

In [41]:
df = df.join(df_output)

In [42]:
df["peak_activations"] = df["#i_1_activations"] + df["#o_1_activations"]

In [43]:
df["total_params"] = df["params"].cumsum()

In [ ]:
df["total_MACS"] = df["MACS"].cumsum()

In [44]:
df.columns

Index(['index', 'name', 'layer_type', 'input_shape', 'input', 'output_shape',
       'dtype', 'params', 'variables', 'weights_shape', 'weights_name',
       'weights_dtype', '#_activations', 'OPS', 'MACS', 'filters', 'trainable',
       'kernel_size', 'strides', 'padding', 'dilation_rate', 'groups',
       'activation', 'use_bias', 'data_format', 'input_1', 'input_2', 'b_i_1',
       'h_i_1', 'w_i_1', 'c_i_1', 'b_i_2', 'h_i_2', 'w_i_2', 'c_i_2',
       '#i_1_activations', '#i_2_activations', 'output_1', 'b_o_1', 'h_o_1',
       'w_o_1', 'c_o_1', '#o_1_activations', 'peak_activations',
       'total_params'],
      dtype='object')

In [45]:
df.head()

,index,name,layer_type,input_shape,input,output_shape,dtype,params,variables,weights_shape,weights_name,weights_dtype,#_activations,OPS,MACS,filters,trainable,kernel_size,strides,padding,dilation_rate,groups,activation,use_bias,data_format,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2,#i_1_activations,#i_2_activations,output_1,b_o_1,h_o_1,w_o_1,c_o_1,#o_1_activations,peak_activations,total_params
0,0,input_1,<class 'keras.engine.input_layer.InputLayer'>,"(None, 96, 96, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 96, 96, 3)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 96, 96, 3)",1,1,96,96,3,1,1,1,1,27648,0,"(None, 96, 96, 3)",1,96,96,3,27648,55296,0
1,1,Conv1,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 96, 96, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,216,1,"(3, 3, 3, 8)",Conv1/kernel:0,<dtype: 'float32'>,18432,995328,497664,8,True,"(3, 3)","(2, 2)",same,"(1, 1)",1,linear,False,channels_last,"(None, 96, 96, 3)",1,1,96,96,3,1,1,1,1,27648,0,"(None, 48, 48, 8)",1,48,48,8,18432,46080,216
2,2,bn_Conv1,<class 'keras.layers.normalization.batch_norma...,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,32,4,(8),bn_Conv1/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 48, 48, 8)",1,1,48,48,8,1,1,1,1,18432,0,"(None, 48, 48, 8)",1,48,48,8,18432,36864,248
3,3,Conv1_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(None, 48, 48, 8)",1,1,48,48,8,1,1,1,1,18432,0,"(None, 48, 48, 8)",1,48,48,8,18432,36864,248
4,4,expanded_conv_depthwise,<class 'keras.layers.convolutional.depthwise_c...,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 48, 48, 8)",float32,72,1,"(3, 3, 8, 1)",expanded_conv_depthwise/depthwise_kernel:0,<dtype: 'float32'>,18432,331776,165888,0,True,"(3, 3)","(1, 1)",same,"(1, 1)",1,linear,False,channels_last,"(None, 48, 48, 8)",1,1,48,48,8,1,1,1,1,18432,0,"(None, 48, 48, 8)",1,48,48,8,18432,36864,320


In [47]:
fig = px.bar(df, x="name", y= "peak_activations", text_auto=".2s",
    title=f"Peak activations per layer - {model_name}")
fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig.show()

In [52]:
fig2 = px.bar(df, x="name", y= "h_i_1", text_auto=".2s",
    title=f"Input height per layer - {model_name}")
#fig2.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange")
#fig2.update_yaxes(autorange="reversed")
#fig2.update_yaxes(rangemode="tozero")
fig2.show()

In [49]:
fig3 = px.bar(df, x="name", y= "c_i_1", text_auto=".2s",
    title=f"Channels per layer - {model_name}")
#fig2.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange")
#fig3.update_yaxes(autorange="reversed")
#fig3.update_yaxes(rangemode="tozero")
# fig3.update_yaxes(
#     range=(0, 1280),
#     constrain='domain'
# )
fig3.show()

In [51]:
fig4 = px.line(df, x="name", y= "total_params",
    title=f"Cumulative parameters per layer - {model_name}")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig4.show()

In [ ]:
fig5 = px.line(df, x="name", y= "total_MACS",
    title=f"Cumulative MACs per layer - {model_name}")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig5.show()

In [55]:
# Generate run ids
id = wandb.util.generate_id()

PROJECT = model_name.split("_")[0]

run = wandb.init(
        # Set the project where this run will be logged
        project=PROJECT, 
        name = model_name,
        id = id, 
        resume="allow",
        sync_tensorboard=True
        )
# Specify the configuration variables
config = wandb.config

#config.batch_size = BATCH_SIZE
#config.dropout =DROPOUT
#config.learn_rate = LR
#config.momentum = MOMENTUM
#config.decay = 1e-6
#config.epochs = EPOCHS
#config.classes = classes
config.id = id
config.architecture = model_name

# Create a table
table = wandb.Table(columns = ["plotly_figure"])

# Create path for Plotly figure
path_to_plotly_html = "./plotly_figure.html"

# Write Plotly figure to HTML
fig.write_html(path_to_plotly_html, auto_play = False) # Setting auto_play to False prevents animated Plotly charts from playing in the table automatically

# Add Plotly figure as HTML file into Table
table.add_data(wandb.Html(path_to_plotly_html))

# Log Table
run.log({"Chart_table": table})

#wandb.log({"Peak activations chart": fig})

wandb.finish()